In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import datetime
import category_encoders as ce

In [44]:
conditions = pd.read_csv("filtered_conditions.csv")
patients = pd.read_csv("filtered_patients.csv")
observations = pd.read_csv("filtered_observations.csv")
care_plans = pd.read_csv("filtered_care_plans.csv")
encounters = pd.read_csv("filtered_encounters.csv")
devices = pd.read_csv("filtered_devices.csv")
supplies = pd.read_csv('filtered_supplies.csv')
procedures = pd.read_csv("filtered_procedures.csv")
medications = pd.read_csv("filtered_medications.csv")
conditionScores = pd.read_csv("condition-scores.csv")
conditionScores = conditionScores.set_index('conditions')

In [45]:
import functools as ft
def conditionScoreAgg(series):
    nD = 0
    for i in series:
        if (conditionScores.at[i, 'category 1'] == 'diabetes'):
            nD+=1
    score = 0 if nD == 0 else (nD*0.2) + 3.8 # diabetes score
    for i in series:
        if (conditionScores.at[i, 'category 1'] != 'diabetes'):
            score += conditionScores.at[i, 'score']  
    return score
conditions = conditions.groupby(['PATIENT'], as_index=False).agg({'DESCRIPTION': conditionScoreAgg})
conditions.columns


Index(['PATIENT', 'DESCRIPTION'], dtype='object')

In [46]:
now = pd.to_datetime('now')
patients['AGE'] = ((now - pd.to_datetime(patients['BIRTHDATE'])).astype('<m8[Y]'))
patients.head(2)
patientRaceEncoder = ce.OneHotEncoder(cols=["RACE"], use_cat_names=True)
encodedPatients = patientRaceEncoder.fit_transform(patients['RACE'])
encodedPatients.head(2)

,RACE_white,RACE_native,RACE_asian,RACE_black,RACE_other
0,1,0,0,0,0
1,1,0,0,0,0


In [47]:
patient_details = patients.drop(patients.columns.difference(['Id', 'AGE', 'ZIP']), axis=1)
patient_details.head(2)

,Id,ZIP,AGE
0,f0f3bc8d-ef38-49ce-a2bd-dfdda982b271,1106.0,3.0
1,067318a4-db8f-447f-8b6e-f2f61e9baaa5,2081.0,4.0


In [56]:
conditions.columns = ['Id', 'Condition_Rank']
patient_details.columns = ['Id', 'Zip', 'Age']
# patient_details.Id = patient_details.Id.astype(str)
# conditions.reset_index()
# print(patient_details.columns)
aggregate = conditions.merge(patient_details, on='Id')
aggregate['Zip'] = aggregate['Zip'].fillna(aggregate.Zip.mean())
aggregate.head(3)

,Id,Condition_Rank,Zip,Age
0,0000b247-1def-417a-a783-41c8682be022,9.0,1997.325872,12.0
1,00049ee8-5953-4edd-a277-b9c1b1a7f16b,15.0,1997.325872,35.0
2,00079a57-24a8-430f-b4f8-a1cf34f90060,24.0,1230.000000,29.0


Create final dataset for model

In [57]:
aggregate.to_csv('clustering_patient_dataset.csv')